# Lab 2
* **Name**: Utkarsh Prakash

## Experiment 1

**Title**: Demonstrate the realization of AND gate and NAND gate using different learning laws for a neuron.

**Objective**: To observe and understand the working of different learning laws for a neuron.

### Part-1: Hebbian Learning Law
#### Hypothesis: 
The Hebbian Learning Law should be able to learn appropriate weights to implement the AND gate and NAND gate. <br />

#### Truth Tables:

**AND Gate** <br />

| $X$ | $Y$ | $Output$ | 
| --- | --- | --- |
| $0$ | $0$ | $0$ |
| $0$ | $1$ | $0$ |
| $1$ | $0$ | $0$ |
| $1$ | $1$ | $1$ |

**NAND Gate** <br />

| $X$ | $Y$ | $Output$ | 
| --- | --- | --- |
| $0$ | $0$ | $1$ |
| $0$ | $1$ | $1$ |
| $1$ | $0$ | $1$ |
| $1$ | $1$ | $0$ |

#### Experimental Description:
1.   Data generation: For training purposes we will consider all the possible combinations of two inputs to the gate along with their desired output i.e. the truth table will be our training data. However, the values of $0$ in the output will be replaced by $-1$. <br />
2.  Objective: We should be able to realize the following:<br />

**AND Gate** <br />

| $a_1$ | $a_2$ | $b$ | 
| --- | --- | --- |
| $-1$ | $-1$ | $-1$ |
| $-1$ | $1$ | $-1$ |
| $1$ | $-1$ | $-1$ |
| $1$ | $1$ | $1$ |

**NAND Gate** <br />

| $a_1$ | $a_2$ | $b$ | 
| --- | --- | --- |
| $-1$ | $-1$ | $1$ |
| $-1$ | $1$ | $1$ |
| $1$ | $-1$ | $1$ |
| $1$ | $1$ | $-1$ |

3. Operations: <br />
   The change in the weight vector is given by:
   <center> $\Delta w_i = \eta f(w^T_i a)a$ </center> <br />
   Therefore, the jth component of $\Delta w_i$ is given by
   <center> $\Delta w_{ij} = \eta f(w^T_ia)a_j = \eta s_i a_j$        where $j = 1, 2, ..., M $ </center> <br />
   where $s_i$ is the output signal of the ith unit.
   The function chosen here is a hard limiting function such as binary function which is defined as follows: <br />
    <center>$f(x) = 1, x > 0$ <br />
    $f(x) = -1, x <=0$</center>
4. Training: We present one training example to the neuron randomly instead of batch processing. At each iteration we normalize the weight vector by dividing the weight vector by its $l2-norm$ i.e., <br/>
    <center>$w = \frac{w}{||w||_2}$</center><br />
    Moreover, the learning rate $\eta$ is close to 1 and the weights are initialized to random values close to zero (using Gaussian Density with mean = 0 and standard deviation = 0.1) <br />
5. Testing: For testing we will use two inputs to the gate and consider all the possible combinations of the input.

#### Implementation

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
def MeanSquaredError(X, y, w):
    '''
        This function calculates the mean squared error over all
        training examples.
    '''
    activation = np.dot(X, w)                 # activation value
    y_pred = np.where(activation > 0, 1, -1)  # output value
    
    return np.sum((y_pred - y)**2)/X.shape[0] # Error calculation

In [3]:
def HebbianLearning(X, y, learning_rate=1.0, norm=2):
    '''
        Function which implements the Hebbian Learning Law.
    '''
    
    # Augmenting the X matrix with 1 for the bias term
    X = np.c_[np.ones((X.shape[0], 1)), X]
    
    # Randomly initilizing weight close to 0
    w = np.random.normal(loc=0.0, scale=0.1, size=X.shape[1])
    
    error = []
    
    # Loop until stopping criterion is met
    while True:
        # Generating random index for random training example
        i = np.random.randint(low=0, high=4, size=1)[0]
        
        activation = np.dot(X[i, :], w)      # Activation value
        s = np.where(activation > 0, -1, 1)       # Output value

        w = w + learning_rate*s*X[i, :].T    # Weight update

        w = w/np.linalg.norm(w, ord=norm)              # Normalizing the weight vector

        error.append(MeanSquaredError(X, y, w))        # error calculation

        # early stopping
        if MeanSquaredError(X, y, w) == 0:
            break
            
    return error, w

In [4]:
# Train vector
X = np.array([[-1, -1],
             [-1, 1],
             [1, -1],
             [1, 1]])

y_and = np.array([-1, -1, -1, 1])        # Train y vector for AND
y_nand = np.array([1, 1, 1, -1])         # Train y vector for NAND

In [7]:
error_and, w_and = HebbianLearning(X, y_and, learning_rate=1.0, norm=2)    # Training AND gate
error_nand, w_nand = HebbianLearning(X, y_nand, learning_rate=1.0, norm=2) # Training NAND gate

print("Number of iterations until convergence for AND gate:", len(error_and))
print("Number of iterations until convergence for NAND gate:", len(error_nand))

Number of iterations until convergence for AND gate: 186
Number of iterations until convergence for NAND gate: 3975


In [117]:
print("Weights for AND gate: ", w_and)
print("Weights for NAND gate: ", w_nand)

Weights for AND gate:  [-0.6812164   0.06868287  0.72885312]
Weights for NAND gate:  [ 0.6887066  -0.72390857 -0.04049192]


#### Testing

In [75]:
def print_result(X_test, result):
    '''
        This function is to print the result of the learned model.
    '''
    print(pd.DataFrame({"a1": X_test[:, 0], "a2": X_test[:, 1], "s": result}).head())

In [76]:
def HebbianPredict(X, w):
    '''
        This function is to predict the outcome for the model
        learned by Hebbian Learning Law.
    '''
    
    # Augmenting the X matrix with 1 for the bias term
    X = np.c_[np.ones((X.shape[0], 1)), X]
    
    activation = np.dot(X, w)              # Activation value
    
    return np.where(activation > 0, 1, -1) # Output Value

In [118]:
# Test vector
X = np.array([[-1, -1],
             [-1, 1],
             [1, -1],
             [1, 1]])

print("AND Result")
print("-------------")
print_result(X, HebbianPredict(X, w_and))     # Testing AND gate
print("\nNAND Result")
print("-------------")
print_result(X, HebbianPredict(X, w_nand))    # Testing NAND gate

AND Result
-------------
   a1  a2  s
0  -1  -1 -1
1  -1   1 -1
2   1  -1 -1
3   1   1  1

NAND Result
-------------
   a1  a2  s
0  -1  -1  1
1  -1   1  1
2   1  -1  1
3   1   1 -1


#### Observations
**Effect of using different norms to normalize the weights:** <br />

**NOTE:** Infinite norm is defined as the $\max_{i}{a_i}$ where $a_i$ the ith component of the vector $a$.

**AND Gate**

| Norm | Number of iterations until convergence|
| --- | --- |
| l1 | 17 |
| l2 | 5181 |
| infinte | Not converging |

**NAND Gate**

| Norm | Number of iterations until convergence|
| --- | --- |
| l1 | 6 |
| l2 | 4299 |
| infinite | Not converging |

**Effect of using different learning rate:** <br />

**AND Gate**

| Learning rate ($\eta$) | Number of iterations until convergence |
| --- | --- |
| 0.1 | 531 |
| 1.0 | 2020 |
| 10.0 | 3 |

**NAND Gate**

| Learning rate ($\eta$) | Number of iterations until convergence |
| --- | --- |
| 0.1 | 1085 |
| 1.0 | 1069 |
| 10.0 | 7 |

#### Conclusion:
* The Hebbian Learning Law is able to learn appropriate weights in order to implement the AND and NAND gate.
* Hebbian Law strengthens the weight if the input and output are of the same sign.
* As we increase the order of the norm used for normalizing the weights, the number of iterations until convergence increases.
* As we change the learning rate ($\eta$), we donot observe any pattern in the number of iterations until convergence.

### Part-2: Correlation Learning Law
#### Hypothesis: 
The Correlation Learning Law should be able to learn appropriate weights to implement the AND gate and NAND gate. <br />

#### Truth Tables:

**AND Gate** <br />

| $X$ | $Y$ | $Output$ | 
| --- | --- | --- |
| $0$ | $0$ | $0$ |
| $0$ | $1$ | $0$ |
| $1$ | $0$ | $0$ |
| $1$ | $1$ | $1$ |

**NAND Gate** <br />

| $X$ | $Y$ | $Output$ | 
| --- | --- | --- |
| $0$ | $0$ | $1$ |
| $0$ | $1$ | $1$ |
| $1$ | $0$ | $1$ |
| $1$ | $1$ | $0$ |

#### Experimental Description:
1.   Data generation: For training purposes we will consider all the possible combinations of two inputs to the gate along with their desired output i.e. the truth table will be our training data. However, the values of $0$ in the output will be replaced by $-1$. <br />
2.  Objective: We should be able to realize the following:<br />

**AND Gate** <br />

| $a_1$ | $a_2$ | $b$ | 
| --- | --- | --- |
| $-1$ | $-1$ | $-1$ |
| $-1$ | $1$ | $-1$ |
| $1$ | $-1$ | $-1$ |
| $1$ | $1$ | $1$ |

**NAND Gate** <br />

| $a_1$ | $a_2$ | $b$ | 
| --- | --- | --- |
| $-1$ | $-1$ | $1$ |
| $-1$ | $1$ | $1$ |
| $1$ | $-1$ | $1$ |
| $1$ | $1$ | $-1$ |

3. Operations: <br />
   The change in the weight vector is given by:
   <center> $\Delta w_i = \eta b_ia$ </center> <br />
   Therefore, the jth component of $\Delta w_i$ is given by
   <center> $\Delta w_{ij} = \eta b_i a_j$        where $j = 1, 2, ..., M $ </center> <br />
   where $s_i$ is the output signal of the ith unit.
   The function chosen here is a hard limiting function such as binary function which is defined as follows: <br />
    <center>$f(x) = 1, x > 0$ <br />
    $f(x) = -1, x <=0$</center>
4. Training: We present one training example to the neuron randomly instead of batch processing. At each iteration we normalize the weight vector by dividing the weight vector by its $l2-norm$ i.e., <br/>
    <center>$w = \frac{w}{||w||_2}$</center><br />
    Moreover, the learning rate $\eta$ is close to 1 and the weights are initialized to random values close to zero (using Gaussian Density with mean = 0 and standard deviation = 0.1) <br />
5. Testing: For testing we will use two inputs to the gate and consider all the possible combinations of the input.

#### Implementation

In [5]:
def CorrelationLearning(X, y, learning_rate=1.0, norm=2):
    '''
        Function which implements the Hebbian Learning Law.
    '''
    
    # Augmenting the X matrix with 1 for the bias term
    X = np.c_[np.ones((X.shape[0], 1)), X]
    
    # Randomly initilizing weight close to 0
    w = np.random.normal(loc=0.0, scale=0.1, size=X.shape[1])
    
    error = []
    
    # Iterating over training examples
    for i in range(X.shape[0]):
        w = w + learning_rate*y[i]*X[i, :].T           # Weight update

        w = w/np.linalg.norm(w, ord=norm)              # Normalizing the weight vector

        error.append(MeanSquaredError(X, y, w))        # Error calculation

        # early stopping
        if MeanSquaredError(X, y, w) == 0:
            break
            
    return error, w

In [6]:
error_and, w_and = CorrelationLearning(X, y_and, learning_rate=1.0, norm=2)    # Training AND gate
error_nand, w_nand = CorrelationLearning(X, y_nand, learning_rate=1.0, norm=2) # Training NAND gate

print("Number of iterations until convergence for AND gate:", len(error_and))
print("Number of iterations until convergence for NAND gate:", len(error_nand))

Number of iterations until convergence for AND gate: 1
Number of iterations until convergence for NAND gate: 1


In [107]:
print("Weights for AND gate: ", w_and)
print("Weights for NAND gate: ", w_nand)

Weights for AND gate:  [-0.60067426  0.42863088  0.67488221]
Weights for NAND gate:  [ 0.59784196 -0.58895713 -0.54379636]


#### Testing

In [97]:
def CorrelationPredict(X, w):
    '''
        This function is to predict the outcome for the model
        learned by Correlation Learning Law.
    '''
    
    # Augmenting the X matrix with 1 for the bias term
    X = np.c_[np.ones((X.shape[0], 1)), X]
    
    activation = np.dot(X, w)               # Activation Value
    
    return np.where(activation > 0, 1, -1)  # Output Value

In [108]:
# Test vector
X = np.array([[-1, -1],
             [-1, 1],
             [1, -1],
             [1, 1]])

print("AND Result")
print("-------------")
print_result(X, CorrelationPredict(X, w_and))
print("\nNAND Result")
print("-------------")
print_result(X, CorrelationPredict(X, w_nand))

AND Result
-------------
   a1  a2  s
0  -1  -1 -1
1  -1   1 -1
2   1  -1 -1
3   1   1  1

NAND Result
-------------
   a1  a2  s
0  -1  -1  1
1  -1   1  1
2   1  -1  1
3   1   1 -1


#### Observations

**Effect of using different learning rate:** <br />

**AND Gate**

| Learning rate ($\eta$) | Convergence |
| --- | --- |
| 0.01 | Not reaching to the solution |
| 0.1 | Not reaching to the solution |
| 1.0 | Reached Solution |
| 1.5 | Reached Solution |

**NAND Gate**

| Learning rate ($\eta$) | Convergence |
| --- | --- |
| 0.01 | Not reaching to the solution |
| 0.1 | Not reaching to the solution |
| 1.0 | Reached Solution |
| 1.5 | Reached Solution |

#### Conclusion:
* The Correlation Learning Law is able to learn appropriate weights in order to implement the AND and NAND gate.
* The Correlation Learning Law updates the weight according to the correlation between input and desired output.
* We observe that only if keep the learning rate ($\eta$) close to 1, then only the model converges to the optimal solution.
* The number of iterations until convergence is significantly less than the Hebbian Learning Law. This can be attributed to fact that Hebbian Law is unsupervised and Correlation Law is supervised.

## Experiment 2

**Title**: Demonstrate the realization of XOR gate using NAND gates learned by using different learning laws for a neuron.

**Objective**: To observe and understand the working of different learning laws for a neuron.

### Part-1: Hebbian Learning Law
#### Hypothesis: 
The NAND gate learned using Hebbian Learning law should be able to implement XOR gate. <br />

#### Truth Tables:

**XOR Gate** <br />

| $X$ | $Y$ | $Output$ | 
| --- | --- | --- |
| $0$ | $0$ | $0$ |
| $0$ | $1$ | $1$ |
| $1$ | $0$ | $1$ |
| $1$ | $1$ | $0$ |

#### Experimental Description:
1.  Objective: We should be able to realize the following:<br />

**XOR Gate** <br />

| $a_1$ | $a_2$ | $b$ | 
| --- | --- | --- |
| $-1$ | $-1$ | $-1$ |
| $-1$ | $1$ | $1$ |
| $1$ | $-1$ | $1$ |
| $1$ | $1$ | $-1$ |

2. Operations: <br />
   We would combine the learned NAND gates in the following order:
   ![NAND_implementation_of_XOR](tmp/NAND_XOR.png)
   <center> Source: https://en.wikipedia.org/wiki/NAND_logic </center>

In [109]:
def HebbianXOR(X, w):
    '''
        This function implements the XOR gate using the NAND gate
        learned using the Hebbian Learning Law.
    '''
    
    # Result for the 1st NAND gate in the circuit
    nand1 = HebbianPredict(X, w)
    
    # Result for the 2nd NAND gate in the circuit
    nand2 = HebbianPredict(np.c_[X[:, 0], nand1], w)
    
    # Result for the 3rd NAND gate in the circuit
    nand3 = HebbianPredict(np.c_[X[:, 1], nand1], w)
    
    # Result for the 4th NAND gate in the circuit
    nand4 = HebbianPredict(np.c_[nand2, nand3], w)
    
    # printing the output
    print_result(X, nand4)

In [119]:
# Test vector
X = np.array([[-1, -1],
             [-1, 1],
             [1, -1],
             [1, 1]])

HebbianXOR(X, w_nand) # Testing XOR gate using Hebbian Law

   a1  a2  s
0  -1  -1 -1
1  -1   1  1
2   1  -1  1
3   1   1 -1


### Part-2: Correlation Learning Law
#### Hypothesis: 
The NAND gate learned using correlation Learning law should be able to implement XOR gate. <br />

#### Truth Tables:

**XOR Gate** <br />

| $X$ | $Y$ | $Output$ | 
| --- | --- | --- |
| $0$ | $0$ | $0$ |
| $0$ | $1$ | $1$ |
| $1$ | $0$ | $1$ |
| $1$ | $1$ | $0$ |

#### Experimental Description:
1.  Objective: We should be able to realize the following:<br />

**XOR Gate** <br />

| $a_1$ | $a_2$ | $b$ | 
| --- | --- | --- |
| $-1$ | $-1$ | $-1$ |
| $-1$ | $1$ | $1$ |
| $1$ | $-1$ | $1$ |
| $1$ | $1$ | $-1$ |

2. Operations: <br />
   We would combine the learned NAND gates in the following order:
   ![NAND_implementation_of_XOR](tmp/NAND_XOR.png)
   <center> Source: https://en.wikipedia.org/wiki/NAND_logic </center>

In [111]:
def CorrelationXOR(X, w):
    '''
        This function implements the XOR gate using the NAND gate
        learned using the Correlation Learning Law.
    '''
    
    # Result for the 1st NAND gate in the circuit
    nand1 = CorrelationPredict(X, w)
    
    # Result for the 2nd NAND gate in the circuit
    nand2 = CorrelationPredict(np.c_[X[:, 0], nand1], w)
    
    # Result for the 3rd NAND gate in the circuit
    nand3 = CorrelationPredict(np.c_[X[:, 1], nand1], w)
    
    # Result for the 4th NAND gate in the circuit
    nand4 = CorrelationPredict(np.c_[nand2, nand3], w)
    
    # printing the output
    print_result(X, nand4)

In [120]:
# Test vector
X = np.array([[-1, -1],
             [-1, 1],
             [1, -1],
             [1, 1]])

CorrelationXOR(X, w_nand) # Testing XOR gate using Correlation Law

   a1  a2  s
0  -1  -1 -1
1  -1   1  1
2   1  -1  1
3   1   1 -1


#### Observations and Conclusion
* By combining learned NAND gates we are able to implement the XOR gates. This means that we can solve non-linearly separable classification problem by using multi-layer neural networks with non-linear processing units. 

## References
* Artificial Neural Networks by B. Yegnanarayana, 1999
* https://www.geeksforgeeks.org/hebbian-learning-rule-with-implementation-of-and-gate/
* Pictures taken from https://en.wikipedia.org/wiki/NAND_logic